In [294]:
import pandas as pd

data = pd.read_csv('../datasets/2.-Urban-Environment/air-quality-nov-2017.csv')

data

,Station,Air Quality,Longitude,Latitude,O3 Hour,O3 Quality,O3 Value,NO2 Hour,NO2 Quality,NO2 Value,PM10 Hour,PM10 Quality,PM10 Value,Generated,Date Time
0,Barcelona - Sants,Good,2.1331,41.3788,NaN,NaN,NaN,0h,Good,84.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104
1,Barcelona - Eixample,Moderate,2.1538,41.3853,0h,Good,1.0,0h,Moderate,113.0,0h,Good,36.0,01/11/2018 0:00,1541027104
2,Barcelona - Gràcia,Good,2.1534,41.3987,0h,Good,10.0,0h,Good,73.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104
3,Barcelona - Ciutadella,Good,2.1874,41.3864,0h,Good,2.0,0h,Good,86.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104
4,Barcelona - Vall Hebron,Good,2.1480,41.4261,0h,Good,7.0,0h,Good,69.0,NaN,NaN,NaN,01/11/2018 0:00,1541027104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5739,Barcelona - Ciutadella,Good,2.1874,41.3864,22h,Good,10.0,22h,Good,57.0,NaN,NaN,NaN,30/11/2018 23:00,1543615502
5740,Barcelona - Vall Hebron,Good,2.1480,41.4261,22h,Good,32.0,22h,Good,31.0,22h,Good,21.0,30/11/2018 23:00,1543615502
5741,Barcelona - Palau Reial,Good,2.1151,41.3875,22h,Good,40.0,22h,Good,20.0,22h,Good,15.0,30/11/2018 23:00,1543615502
5742,Barcelona - Poblenou,Good,2.2045,41.4039,NaN,NaN,NaN,22h,Good,70.0,22h,Good,25.0,30/11/2018 23:00,1543615502


In [295]:
#dropping non-PM10-related columns

pm = data.drop(columns=['O3 Hour','O3 Quality','O3 Value','NO2 Hour','NO2 Quality','NO2 Value','Date Time','Longitude','Latitude'])

pm

,Station,Air Quality,PM10 Hour,PM10 Quality,PM10 Value,Generated
0,Barcelona - Sants,Good,NaN,NaN,NaN,01/11/2018 0:00
1,Barcelona - Eixample,Moderate,0h,Good,36.0,01/11/2018 0:00
2,Barcelona - Gràcia,Good,NaN,NaN,NaN,01/11/2018 0:00
3,Barcelona - Ciutadella,Good,NaN,NaN,NaN,01/11/2018 0:00
4,Barcelona - Vall Hebron,Good,NaN,NaN,NaN,01/11/2018 0:00
...,...,...,...,...,...,...
5739,Barcelona - Ciutadella,Good,NaN,NaN,NaN,30/11/2018 23:00
5740,Barcelona - Vall Hebron,Good,22h,Good,21.0,30/11/2018 23:00
5741,Barcelona - Palau Reial,Good,22h,Good,15.0,30/11/2018 23:00
5742,Barcelona - Poblenou,Good,22h,Good,25.0,30/11/2018 23:00


In [311]:
#Removing NaN rows

nantodrop = pm[pm['PM10 Value'].isna() == True].index.tolist()
pm2 = pm.drop(nantodrop)
pm2.reset_index(drop=True,inplace=True)

pm2['PM10 Value'].value_counts().sum()

3647

In [228]:
#Removing "h" from PM10 Hour column elements in a new column

pm2['time'] = list(map(lambda x: int(re.findall(r"\d+",x)[0]),pm2['PM10 Hour']))
pm2.rename(columns={'time':'Time (h)'},inplace=True)
pm2.drop(columns=['PM10 Hour'],inplace=True)
pm2

,Station,Air Quality,PM10 Quality,PM10 Value,Generated,Time (h)
0,Barcelona - Eixample,Moderate,Good,36.0,01/11/2018 0:00,0
1,Barcelona - Palau Reial,Good,Good,23.0,01/11/2018 0:00,23
2,Barcelona - Poblenou,Good,Good,32.0,01/11/2018 0:00,23
3,Barcelona - Observ Fabra,Good,Good,25.0,01/11/2018 0:00,23
4,Barcelona - Eixample,Good,Good,35.0,01/11/2018 1:00,1
...,...,...,...,...,...,...
3642,Barcelona - Gràcia,Good,Good,22.0,30/11/2018 23:00,22
3643,Barcelona - Vall Hebron,Good,Good,21.0,30/11/2018 23:00,22
3644,Barcelona - Palau Reial,Good,Good,15.0,30/11/2018 23:00,22
3645,Barcelona - Poblenou,Good,Good,25.0,30/11/2018 23:00,22


In [290]:
# Analyzing how it changes throughout the day

roun = lambda x: np.round(np.mean(x),decimals=2)
hourly = pm2.groupby('Time (h)')['PM10 Value'].agg(['min','max','mean']).round(2).sort_values('Time (h)')
hourly_df1 = hourly.iloc[:12,:]
hourly_df2 = hourly.iloc[12:,:]

,min,max,mean
Time (h),,,
0,2.0,42.0,16.07
1,3.0,43.0,17.23
2,2.0,39.0,16.34
3,2.0,44.0,17.23
4,3.0,44.0,16.81
5,2.0,44.0,16.88
6,2.0,44.0,16.59
7,2.0,43.0,16.66
8,2.0,42.0,16.46


In [291]:
hourly_df1

,min,max,mean
Time (h),,,
0,2.0,42.0,16.07
1,3.0,43.0,17.23
2,2.0,39.0,16.34
3,2.0,44.0,17.23
4,3.0,44.0,16.81
5,2.0,44.0,16.88
6,2.0,44.0,16.59
7,2.0,43.0,16.66
8,2.0,42.0,16.46


In [289]:
hourly_df2

,min,max,mean
Time (h),,,
12,3.0,41.0,16.34
13,3.0,42.0,16.93
14,3.0,42.0,16.55
15,2.0,43.0,16.53
16,2.0,43.0,16.51
17,2.0,42.0,16.01
18,2.0,41.0,16.81
19,3.0,41.0,16.73
20,2.0,41.0,16.51


In [230]:
#Analyzing the difference between Stations

pm2.groupby('Station')['PM10 Value'].agg(['min','max','mean']).round(2).sort_values('mean')

,min,max,mean
Station,,,
Barcelona - Observ Fabra,5.0,25.0,10.77
Barcelona - Vall Hebron,2.0,32.0,13.91
Barcelona - Palau Reial,6.0,29.0,14.53
Barcelona - Gràcia,7.0,31.0,16.82
Barcelona - Poblenou,7.0,38.0,20.54
Barcelona - Eixample,11.0,44.0,22.78


In [298]:
# to see full table

pd.set_option('display.max_rows', 50)

In [232]:
#Analyzing the hourly change for every station

display(pm2.groupby(['Station','Time (h)'])['PM10 Value'].agg(['min','max','mean']).round(2))

min   max   mean
Station                 Time (h)                   
Barcelona - Eixample    0         12.0  42.0  22.43
                        1         11.0  43.0  22.97
                        2         11.0  39.0  21.93
                        3         11.0  44.0  23.70
                        4         11.0  44.0  23.34
...                                ...   ...    ...
Barcelona - Vall Hebron 19         3.0  32.0  14.42
                        20         2.0  25.0  13.27
                        21         2.0  32.0  13.82
                        22         3.0  31.0  15.04
                        23         2.0  31.0  13.57

[144 rows x 3 columns]

In [233]:
df_nov = pm2[pm2['Generated'].str.contains('\d{2}/11/2018')].groupby(['Generated'])
df_nov.head(10)

,Station,Air Quality,PM10 Quality,PM10 Value,Generated,Time (h)
0,Barcelona - Eixample,Moderate,Good,36.0,01/11/2018 0:00,0
1,Barcelona - Palau Reial,Good,Good,23.0,01/11/2018 0:00,23
2,Barcelona - Poblenou,Good,Good,32.0,01/11/2018 0:00,23
3,Barcelona - Observ Fabra,Good,Good,25.0,01/11/2018 0:00,23
4,Barcelona - Eixample,Good,Good,35.0,01/11/2018 1:00,1
...,...,...,...,...,...,...
3642,Barcelona - Gràcia,Good,Good,22.0,30/11/2018 23:00,22
3643,Barcelona - Vall Hebron,Good,Good,21.0,30/11/2018 23:00,22
3644,Barcelona - Palau Reial,Good,Good,15.0,30/11/2018 23:00,22
3645,Barcelona - Poblenou,Good,Good,25.0,30/11/2018 23:00,22


In [241]:
#Separate into one dataframe per day of the week
monday = pm2[pm2['Generated'].str.contains('05/11/2018|12/11/2018|19/11/2018|26/11/2018')]
tuesday = pm2[pm2['Generated'].str.contains('06/11/2018|13/11/2018|20/11/2018|27/11/2018')]
wednesday = pm2[pm2['Generated'].str.contains('07/11/2018|14/11/2018|21/11/2018|28/11/2018')]
thursday = pm2[pm2['Generated'].str.contains('01/11/2018|08/11/2018|15/11/2018|22/11/2018|29/11/2018')]
friday = pm2[pm2['Generated'].str.contains('02/11/2018|09/11/2018|16/11/2018|23/11/2018|30/11/2018')]
saturday = pm2[pm2['Generated'].str.contains('03/11/2018|10/11/2018|17/11/2018|24/11/2018')]
sunday = pm2[pm2['Generated'].str.contains('04/11/2018|11/11/2018|18/11/2018|25/11/2018')]

monday

,Station,Air Quality,PM10 Quality,PM10 Value,Generated,Time (h)
425,Barcelona - Eixample,Good,Good,19.0,05/11/2018 0:00,0
426,Barcelona - Vall Hebron,Good,Good,2.0,05/11/2018 0:00,0
427,Barcelona - Palau Reial,Good,Good,16.0,05/11/2018 0:00,0
428,Barcelona - Poblenou,Good,Good,19.0,05/11/2018 0:00,0
429,Barcelona - Observ Fabra,Good,Good,14.0,05/11/2018 0:00,0
...,...,...,...,...,...,...
3099,Barcelona - Eixample,Good,Good,13.0,26/11/2018 23:00,22
3100,Barcelona - Gràcia,Good,Good,13.0,26/11/2018 23:00,22
3101,Barcelona - Vall Hebron,Good,Good,10.0,26/11/2018 23:00,22
3102,Barcelona - Palau Reial,Good,Good,10.0,26/11/2018 23:00,23


In [252]:
#Calculating average contamination per day

mon_mean = monday['PM10 Value'].mean()
tue_mean = tuesday['PM10 Value'].mean()
wed_mean = wednesday['PM10 Value'].mean()
thu_mean = thursday['PM10 Value'].mean()
fri_mean = friday['PM10 Value'].mean()
sat_mean = saturday['PM10 Value'].mean()
sun_mean = sunday['PM10 Value'].mean()

In [262]:
# creating DataFrame for weekdays
weekdays = [['Monday', mon_mean], ['Tuesday', tue_mean], ['Wednesday', wed_mean], ['Thursday', thu_mean], ['Friday', fri_mean], ['Saturday', sat_mean], ['Sunday', sun_mean]]

weekdf = pd.DataFrame(weekdays,columns=['Weekdays','PM10 Mean']).set_index('Weekdays')
weekdf

,PM10 Mean
Weekdays,
Monday,16.427984
Tuesday,17.660338
Wednesday,17.319192
Thursday,18.869707
Friday,19.143590
Saturday,14.234818
Sunday,11.541082


In [272]:
#Separate into one dataframe per week
week_1 = pm2[pm2['Generated'].str.contains('01/11/2018|02/11/2018|03/11/2018|04/11/2018|05/11/2018|06/11/2018|07/11/2018')]
week_2 = pm2[pm2['Generated'].str.contains('08/11/2018|09/11/2018|10/11/2018|11/11/2018|12/11/2018|13/11/2018|14/11/2018')]
week_3 = pm2[pm2['Generated'].str.contains('15/11/2018|16/11/2018|17/11/2018|18/11/2018|19/11/2018|20/11/2018|21/11/2018')]
week_4 = pm2[pm2['Generated'].str.contains('22/11/2018|23/11/2018|24/11/2018|25/11/2018|26/11/2018|27/11/2018|28/11/2018')]
#week_5 = pm2[pm2['Generated'].str.contains('29/11/2018|30/11/2018')]
week_1

,Station,Air Quality,PM10 Quality,PM10 Value,Generated,Time (h)
0,Barcelona - Eixample,Moderate,Good,36.0,01/11/2018 0:00,0
1,Barcelona - Palau Reial,Good,Good,23.0,01/11/2018 0:00,23
2,Barcelona - Poblenou,Good,Good,32.0,01/11/2018 0:00,23
3,Barcelona - Observ Fabra,Good,Good,25.0,01/11/2018 0:00,23
4,Barcelona - Eixample,Good,Good,35.0,01/11/2018 1:00,1
...,...,...,...,...,...,...
757,Barcelona - Eixample,Good,Good,21.0,07/11/2018 23:00,23
758,Barcelona - Vall Hebron,Good,Good,9.0,07/11/2018 23:00,23
759,Barcelona - Palau Reial,Good,Good,11.0,07/11/2018 23:00,23
760,Barcelona - Poblenou,Good,Good,19.0,07/11/2018 23:00,23


In [273]:
#Calculating average contamination per day

w1_mean = week_1['PM10 Value'].mean()
w2_mean = week_2['PM10 Value'].mean()
w3_mean = week_3['PM10 Value'].mean()
w4_mean = week_4['PM10 Value'].mean()
#w5_mean = week_5['PM10 Value'].mean()

In [274]:
# creating DataFrame for weeks
weeks = [['Week 1', w1_mean], ['Week 2', w2_mean], ['Week 3', w3_mean], ['Week 4', w4_mean]] #['Week 5', w5_mean]]

weeksdf = pd.DataFrame(weeks,columns=['Weeks','PM10 Mean']).set_index('Weeks')
weeksdf

,PM10 Mean
Weeks,
Week 1,13.481627
Week 2,19.594096
Week 3,17.355828
Week 4,13.561404


In [59]:
pm.groupby('Station')['PM10 Quality'].value_counts()

Station                   PM10 Quality
Barcelona - Eixample      Good            655
                          Moderate         63
Barcelona - Gràcia        Good            211
Barcelona - Observ Fabra  Good            693
                          --               21
Barcelona - Palau Reial   Good            696
                          --               22
Barcelona - Poblenou      Good            667
                          --               32
                          Moderate         17
Barcelona - Vall Hebron   Good            645
Name: PM10 Quality, dtype: int64

In [25]:
pm.groupby('Station')['PM10 Value'].describe().sort_values('mean')

#Observatori Fabra and Vall Hebron are further away from the port
#Quite some distance
#Then Palau and closely followed by Gracia
#Then Sants

,count,mean,std,min,25%,50%,75%,max
Station,,,,,,,,
Barcelona - Observ Fabra,693.0,10.773449,4.706629,5.0,7.0,9.0,13.0,25.0
Barcelona - Vall Hebron,645.0,13.910078,6.795914,2.0,9.0,12.0,18.0,32.0
Barcelona - Palau Reial,696.0,14.527299,5.583568,6.0,10.0,13.0,18.0,29.0
Barcelona - Gràcia,211.0,16.815166,6.026841,7.0,12.0,17.0,21.0,31.0
Barcelona - Poblenou,684.0,20.540936,7.870088,7.0,15.0,19.0,27.0,38.0
Barcelona - Eixample,718.0,22.781337,8.454874,11.0,16.0,22.0,29.0,44.0
Barcelona - Ciutadella,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Barcelona - Sants,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
